Project Luther: Ebertron
--
Ozzie Liu - ozzie@ozzieliu.com

Part 2/3 - Review wrangling and processing 

This IPython notebook documents the process of wrangling and cleaning data before it can be used for regression and exploratory data analysis. Results are pickled for use in the next part

### Process:
1. <a href='#1'>Open and merge OMDB flatfile</a>
2. <a href='#2'>Open and merge Rotten Tomatoes file with ratings.</a>
3. <a href='#3'>Create dummy variables for genres</a>

### Packages:
- numpy: for some numerical analysis
- pandas: for data frames
- pickle: to pickle things


In [1]:
import pandas as pd
import pickle
import numpy as np

In [ ]:
## Load the metadata scraped in Part 1
review_metadata = pickle.load(open('ebert_reviews.pkl', 'rb'))

# 1. Open and merge OMDB flatfile
<a id='1'></a>
See <a href='http://www.omdbapi.com/'>OMDB</a> for more information on obtaining the flatfile

In [ ]:
## Load the OMDB flatfile
omdb = pd.read_csv('omdb0116/omdbMovies.txt', sep = '\t', error_bad_lines=False)

# omdb = pickle.load(open('omdb.pkl', 'rb'))

In [ ]:
## That's a lot of movies. We're not concerned with all of those. I start by getting rid of movies before 1980s
omdb = omdb[omdb['Year'] > 1980]

## Define a helper function to strip the 'min' from runtime column
def strip_min(time):
    try:
        return int(time.split()[0])
    except:
        return time
## and apply strip_min() to the entire column.
omdb['Runtime'] = omdb['Runtime'].apply(strip_min, 1)

## Keep only movies with runtime longer than 30 minutes
omdb = omdb[omdb['Runtime'] >= 30]
## and if rating is not adult
omdb = omdb[omdb['Rating'] != 'X']
## and has imdbVotes
omdb = omdb[np.isfinite(omdb['imdbVotes'])]
## that's more than 1200 votes
omdb = omdb[omdb['imdbVotes'] > 1200]
## and also has a metacritic score
omdb = omdb[np.isfinite(omdb['Metacritic'])]

In [ ]:
## Now merge the two lists with a left join on the movie title
review_df = review_metadata.merge(omdb, how='left', on='Title')
## Select the columns that I want
review_df = review_df[['Title', 'EbertStars', 'Year_x', 'URL', 'ID', 'imdbID', 'Genre', 'Metacritic', 'imdbRating', 'imdbVotes', 'Country', 'Awards']]

## Note: because we're joining on movie titles, there may be duplicate rows for movies with the same title.
## This step is omitted 

# 2. Open and merge Rotten Tomatoes file with ratings.
<a id='2'></a>

In [ ]:
## Open tab separated CSV file
tomatoes = pd.read_csv('omdb0116/tomatoes.txt', sep='\t')
## Since we're using IMDB's movie ID to join RT ratings, I remove any movies that doesn't have ID
review_df = review_df[np.isfinite(review_df['ID'])]
## Merge with tomatoes
review_df = review_df.merge(tomatoes, how = 'left', on = 'ID')

# 3. Create dummy variables for all genres
<a id='3'></a>
Take the current string of genres and make it into a true list first.

In [19]:
movie_df = pickle.load(open('movie_df.pkl', 'rb'))

In [20]:
"""
Take a string of genre and split them into a list of genres for easy searching later
Input: string of genres listed
Output: list of genres
"""

def list_genres(my_string):
    return my_string.replace(',', '').split()

In [21]:
## Apply the helper function and get the set of genres.
movie_df['Genre'] = movie_df['Genre'].apply(list_genres, 1)

In [22]:
genre_list = list()
for gl in movie_df['Genre']:
    genre_list = genre_list + gl
# genre_set = set(movie_df['Genre'])


In [24]:
genre_set = set(genre_list)
print genre_set

In [27]:
"""
Make dummy variables for all the genres.
Input is the row in which we'll apply in the dataFrame
Output is a Series of columns corresponding to that genre. 1 indicated that it is of that genre. 
0 indicates that it's not
"""

def dummy_genres(row):
    genrelist = ['Action','Adventure','Animation','Biography','Comedy','Crime','Documentary','Drama','Family','Fantasy',
          'History','Horror','Music','Musical','Mystery','News','Romance','Sci-Fi','Short','Sport','Thriller','War','Western']
    
    result = [0]*len(genrelist)
    
    for index, gen in enumerate(genrelist):
        result[index] = int(gen in row['Genre'])
            
    return pd.Series({'Action':result[0],'Adventure':result[1],'Animation':result[2],'Biography':result[3],'Comedy':result[4],
                      'Crime':result[5],'Documentary':result[6],'Drama':result[7],'Family':result[8],'Fantasy':result[9],
                      'History':result[10],'Horror':result[11],'Music':result[12],'Musical':result[13],'Mystery':result[14],
                      'News':result[15],'Romance':result[16],'Sci-Fi':result[17],'Short':result[18],'Sport':result[19],
                      'Thriller':result[20],'War':result[21],'Western':result[22]})

In [28]:
## Apply to create new columns for those genre.
movie_df[['Action','Adventure','Animation','Biography','Comedy','Crime','Documentary','Drama','Family','Fantasy',
          'History','Horror','Music','Musical','Mystery','News','Romance',
          'Sci-Fi','Short','Sport','Thriller','War','Western']] = movie_df.apply(dummy_genres, 1)

In [38]:
## Considering that Ebert watches a lot foreign films, we should add this as a genre.
## I define foreign as any film that was not made in an english speaking country
def split_country(s):
    try:
        return s.split(',')
    except:
        return []

movie_df['Country'] = movie_df['Country'].apply(split_country, 1)

In [39]:
def check_foreign(row):
    country = set(row['Country'])
    english = set(['USA', 'UK', 'Canada'])

    if len(english & country) > 0:
        return 0
    else:
        return 1
    
movie_df['foreign'] = movie_df.apply(check_foreign, 1)

In [42]:
## Now remove some columns that I don't care for anymore:
movie_df = movie_df.drop(['Year_x', 'URL', 'ID', 'imdbVotes', 'Awards', 'Country', 'Image', 'Meter', 'Reviews', 'Fresh',
                         'Rotten', 'Consensus', 'userMeter', 'DVD', 'BoxOffice', 'Production', 'Website', 'lastUpdated'], 1)

In [45]:
## Pickle and store file to use in regression and EDA Notebook
pickle.dump(movie_df, open('movie_reviews_full.pkl', 'wb'))